# ephax Quickstart Tutorial

Welcome! This notebook walks through a minimal workflow for the analyzer-first electrophysiology toolkit. You'll learn how to prepare a dataset, choose reference electrodes, and run a few of the provided analyzers on synthetic data.

## Prerequisites

- Python 3.9+ with Jupyter installed (`pip install jupyterlab`)
- The `ephax` package available in your environment (`pip install -e .` from the repository root)
- MEA spike recordings in `.h5` or `.npz` form — we will start with a synthetic dataset you can run anywhere

In [ ]:

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

from ephax import RestingActivityDataset, PrepConfig, Recording
from ephax.analyzers import IFRAnalyzer
from ephax.analyzers.ifr import IFRConfig
from ephax.analyzers.firing_distance import FiringDistanceAnalyzer
from ephax.analyzers.cofiring_temporal import CofiringTemporalAnalyzer, CofiringTemporalConfig

plt.rcParams["figure.dpi"] = 120
np.random.seed(0)


## Building a Synthetic Dataset

If you do not have access to raw recordings yet, you can still explore the API with generated spike trains. The helper below creates a grid of electrodes, draws Poisson spike trains, and adds brief synchronous bursts across a few electrodes so that the downstream analyzers have interesting structure to work with.

In [ ]:

from typing import Tuple

def simulate_recording(
    seed: int,
    duration_s: float = 60.0,
    grid_side: int = 4,
    base_rate_range: Tuple[float, float] = (5.0, 20.0),
) -> Recording:
    """Create a Recording with synthetic spikes and square-grid layout."""
    rng = np.random.default_rng(seed)
    n_electrodes = grid_side ** 2
    channels = np.arange(n_electrodes, dtype=np.int32)
    electrodes = np.arange(100, 100 + n_electrodes, dtype=np.int32)
    pitch_um = 60.0
    layout = {
        "channel": channels,
        "electrode": electrodes,
        "x": ((channels % grid_side) * pitch_um).astype(np.float32),
        "y": ((channels // grid_side) * pitch_um).astype(np.float32),
    }

    base_rates = rng.uniform(base_rate_range[0], base_rate_range[1], size=n_electrodes)
    burst_times = rng.uniform(5.0, duration_s - 5.0, size=12)

    all_times = []
    all_channels = []
    all_electrodes = []
    all_amplitudes = []

    for idx in range(n_electrodes):
        baseline_count = rng.poisson(base_rates[idx] * duration_s)
        baseline = rng.uniform(0.0, duration_s, size=baseline_count)
        if idx < grid_side:
            correlated = burst_times + rng.normal(scale=0.004, size=burst_times.shape)
            correlated = np.clip(correlated, 0.0, duration_s)
            spikes_t = np.concatenate([baseline, correlated])
        else:
            spikes_t = baseline
        spikes_t = np.sort(spikes_t)
        if spikes_t.size == 0:
            continue
        all_times.append(spikes_t.astype(np.float32))
        all_channels.append(np.full(spikes_t.size, channels[idx], dtype=np.int32))
        all_electrodes.append(np.full(spikes_t.size, electrodes[idx], dtype=np.int32))
        all_amplitudes.append(rng.normal(loc=45.0, scale=5.0, size=spikes_t.size).astype(np.float32))

    if not all_times:
        raise RuntimeError("Synthetic recording is empty; adjust simulation parameters.")

    spikes = {
        "time": np.concatenate(all_times),
        "channel": np.concatenate(all_channels),
        "electrode": np.concatenate(all_electrodes),
        "amplitude": np.concatenate(all_amplitudes),
    }

    return Recording(
        spikes=spikes,
        layout=layout,
        start_time=0.0,
        end_time=float(duration_s),
        sf=10_000.0,
    )


In [ ]:

recordings = [simulate_recording(0), simulate_recording(42)]
ds = RestingActivityDataset(recordings=recordings, sf=10_000.0)

print(f"Dataset contains {len(ds.recordings)} recordings")
for idx, rec in enumerate(ds.recordings):
    spike_count = len(rec.spikes["time"])
    electrode_count = np.unique(rec.spikes["electrode"]).size
    duration = rec.end_time - rec.start_time
    print(f"  Recording {idx}: {spike_count} spikes across {electrode_count} electrodes in {duration:.1f} s")


## Selecting Reference Electrodes with `PrepConfig`

Analyzers expect a list of reference electrodes per recording. `PrepConfig` lets you request them by activity threshold, by rank, or by providing an explicit list. Here we pick the top few most active electrodes within each recording window.

In [ ]:

prep_cfg = PrepConfig(mode="top", top_start=0, top_stop=6, verbose=False)
refs_per_recording = ds.select_ref_electrodes(prep_cfg)
for idx, refs in enumerate(refs_per_recording):
    preview = ", ".join(map(str, refs[:5])) + ("…" if refs.size > 5 else "")
    print(f"Recording {idx}: selected {refs.size} reference electrodes ({preview})")


## Instantaneous Firing Rate (IFR) Analysis

`IFRAnalyzer` pools spikes across recordings, computes instantaneous firing rate samples, optionally fits a Gaussian mixture model, and can render per-recording heatmaps. We reuse our `PrepConfig` so the analyzer focuses on the same reference electrodes.

In [ ]:

ifr_cfg = IFRConfig(log_scale=True, hist_bins=60, overlay_gmm=True, time_grid_hz=200.0, max_time_points=2_000)
ifr_analyzer = IFRAnalyzer.from_dataset(ds, config=ifr_cfg, selection_prep_config=prep_cfg)
peaks = ifr_analyzer.peaks()
print(f"Collected {peaks.values.size} IFR samples")
if peaks.peaks_hz.size:
    print("GMM peak locations (Hz):", np.round(peaks.peaks_hz, 2))
else:
    print("No peaks detected — consider widening the selection window.")


In [ ]:

fig, ax = ifr_analyzer.plot_histogram()
fig


In [ ]:

ts_figs = ifr_analyzer.plot_timeseries()
for fig, axes in ts_figs:
    display(fig)


## Distance-Dependent Firing Statistics

`FiringDistanceAnalyzer` aggregates firing or co-firing metrics as a function of inter-electrode distance. Even with synthetic data, you can inspect whether firing rates decay with distance, and overlay the theoretical synergy curve derived from the detected IFR peaks.

In [ ]:

fd_analyzer = FiringDistanceAnalyzer(ds, selection_prep_config=prep_cfg)
rate_vs_distance = fd_analyzer.avg_rate_vs_distance()
fig, _ = fd_analyzer.plot_rate_with_synergy(rate_vs_distance, show_exponential_fit=False)
fig


## Temporal Co-firing Heatmap

`CofiringTemporalAnalyzer` summarizes how co-firing probability changes with inter-electrode distance and temporal delay. The visualization below averages across reference electrodes and recordings using the same selection we defined earlier.

In [ ]:

cft_cfg = CofiringTemporalConfig(start_ms=-20, stop_ms=120, step_ms=20, normalize=False)
cft_analyzer = CofiringTemporalAnalyzer(ds, cft_cfg, selection_prep_config=prep_cfg)
fig, ax = cft_analyzer.plot_avg_cofiring_heatmap()
fig


### Bundled Example Recordings

The repository ships with short `.raw.h5` snippets under `ephax/data`. Load them by pointing `file_info` at that folder:

```python
from pathlib import Path
from ephax import RestingActivityDataset

repo_root = Path.cwd()  # run the notebook from the repository root
file_info = [
    ("ephax/data", "well_0.raw.h5", 0, 600, 0),
    ("ephax/data", "well_1.raw.h5", 0, 600, 1),
    ("ephax/data", "well_2_3.raw.h5", 0, 600, 2),
    ("ephax/data", "well_2_3.raw.h5", 0, 600, 3),
    ("ephax/data", "well_4.raw.h5", 0, 600, 4),
    ("ephax/data", "well_5.raw.h5", 0, 600, 5),
]

def resolve(entry):
    folder, filename, start, end, well = entry
    folder_path = repo_root / folder
    return (folder_path.as_posix(), filename, start, end, well)

resolved_info = [resolve(item) for item in file_info]
ds = RestingActivityDataset.from_file_info(resolved_info, source="h5", min_amp=0)
```

If you launch Jupyter from elsewhere, adjust `repo_root` accordingly or provide absolute paths in `file_info`.


## Working with Your Own Recordings

Replace the synthetic dataset with real recordings by preparing a `file_info` list and calling `RestingActivityDataset.from_file_info`:

```python
file_info = [
    ("2407", "control_0.raw.h5", 0, 600, 0),  # (folder/div, filename, start_s, end_s, well)
    ("2407", "control_1.raw.h5", 0, 600, 1),
]
ds = RestingActivityDataset.from_file_info(file_info, source="h5", min_amp=0)
```

After loading, keep the rest of the notebook unchanged: reuse `PrepConfig`, instantiate analyzers, and iterate on configuration parameters as needed. Consult `README.md` for additional analyzers (stability, DCT, GIF generation) and advanced options such as permutation controls.